# Setup

In [ ]:
%env PIP_INDEX_URL=https://pypi.org/simple
%pip install -U langgraph langsmith
%pip install -U "langchain[aws]"

# Model

In [ ]:
from langchain.chat_models import init_chat_model # type: ignore

# Follow the steps here to configure your credentials:
# https://docs.aws.amazon.com/bedrock/latest/userguide/getting-started.html

llm = init_chat_model(
    "anthropic.claude-3-haiku-20240307-v1:0",
    model_provider="bedrock_converse",
)

# Create a StateGraph

In [ ]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START # type: ignore
from langgraph.graph.message import add_messages # type: ignore


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

In [ ]:
def chatbot(state: State):
    print("Chatbot invoked:", state["messages"])
    return {"messages": [llm.invoke(state["messages"])]}


# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile()

# Visualize the graph (optional)

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

# Run the chatbot

In [ ]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)


while True:
    try:
        user_input = input("User: ")
        print("User: " + user_input)
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        # user_input = "What do you know about LangGraph?"
        # print("User: " + user_input)
        # stream_graph_updates(user_input)
        break

User: hello
Chatbot invoked [HumanMessage(content='hello', additional_kwargs={}, response_metadata={}, id='0f41ddc1-090c-4cfb-ab57-d6106a121f35')]
value_messages: [AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': '95ec63aa-3915-4af0-bb3f-050eacf11e96', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 14 May 2025 03:28:31 GMT', 'content-type': 'application/json', 'content-length': '214', 'connection': 'keep-alive', 'x-amzn-requestid': '95ec63aa-3915-4af0-bb3f-050eacf11e96'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': [309]}, 'model_name': 'anthropic.claude-3-haiku-20240307-v1:0'}, id='run--0a32e8fd-75e1-4b04-98cd-e4fade0a10f1-0', usage_metadata={'input_tokens': 8, 'output_tokens': 12, 'total_tokens': 20, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}})]
Assistant: Hello! How can I assist you today?
User: how are you
Chatbot invoked [HumanMessage(content='how 